In [1]:
# source: https://medium.com/grabngoinfo/install-pyspark-3-on-google-colab-the-easy-way-577ec4a2bcd8
# Download Java Virtual Machine (JVM)
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
# Unzip the file
!tar xf spark-3.3.1-bin-hadoop3.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.3.1-bin-hadoop3'

# Install library for finding Spark
!pip install -q findspark
# Import the libary
import findspark
# Initiate findspark
findspark.init()
# Check the location for Spark
findspark.find()

# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

In [2]:
# Connect to Google Drive
# Need manual permission to access your drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
cd gdrive/MyDrive

/content/gdrive/MyDrive


In [54]:
# Algorithm by Kees Voorintholt
# https://github.com/Keesiev7/MSTforDenseGraphs
# Small modifications/bug fixes have been made
import math
import csv
import random
import scipy.spatial
import numpy as np
import matplotlib.pyplot as plt
import folium

from datetime import datetime
from sklearn.datasets import make_circles, make_moons, make_blobs, make_swiss_roll, make_s_curve
from pyspark import SparkConf, SparkContext

import matplotlib.pyplot as plt
import networkx as nx

import json
import csv
import random
import numpy as np
import scipy
import pandas as pd

def get_edge_weight(num_clusters=5, sigma=1, mu=None):
    if mu is None:
        mu = [5, 10, 15, 20, 25]
    edge_cluster = np.ceil(random.uniform(0, num_clusters))
    return np.random.normal(mu[edge_cluster], sigma)

class DataReader:

    def __init__(self):
        pass

    def create_distance_matrix(self, dataset, full_dm=False):
        """
        Creates the distance matrix for a dataset with only vertices. Also adds the edges to a dict.
        :param dataset: dataset without edges
        :return: distance matrix, a dict of all edges and the total number of edges
        """
        vertices = []
        size = 0
        three_d = False
        for line in dataset:
            if len(line) == 2:
                vertices.append([line[0], line[1]])
            elif len(line) == 3:
                vertices.append([line[0], line[1], line[2]])
                three_d = True
        if three_d:
            max_weight = 0
            dict = {}
            for i in range(len(dataset)):
                dict2 = {}
                for j in range(i + 1, len(dataset)):
                    value = np.sqrt(np.sum(np.square(dataset[i] - dataset[j])))
                    max_weight = max(value, max_weight)
                    dict2[j] = value
                    size += 1
                dict[i] = dict2
        else:
            d_matrix = scipy.spatial.distance_matrix(vertices, vertices, threshold=1000000)
            dict = {}
            max_weight = 0
            # Run with less edges
            for i in range(len(d_matrix)):
                dict2 = {}
                if full_dm:
                    for j in range(len(d_matrix)):
                        if i != j:
                            size += 1
                            max_weight = max(d_matrix[i][j], max_weight)
                            dict2[j] = d_matrix[i][j]
                    dict[i] = dict2
                else:
                    for j in range(i, len(d_matrix)):
                        if i != j:
                            size += 1
                            max_weight = max(d_matrix[i][j], max_weight)
                            dict2[j] = d_matrix[i][j]
                    dict[i] = dict2
        return dict, size, vertices, max_weight

    def read_vertex_list(self, file_location):
        vertices = []
        with open(file_location) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            for row in csv_reader:
                vertices.append((float(row[0]), float(row[1])))
        return vertices, len(vertices)

    def read_csv_columns(self, file_location, column_names):
        df = pd.read_csv(file_location, usecols=column_names)
        V = []
        for index, line in df.iterrows():
            V.append((float(line[0]), float(line[1])))
        return V, len(V)

def get_key(item):
    """
    returns the sorting criteria for the edges. All edges are sorted from small to large values
    :param item: one item
    :return: returns the weight of the edge
    """
    return item[2]

def create_clusters(clusters, dict_edges, flight_data = False):
    i = 0
    while i < len(clusters):
        pop = False
        for j in range(i):
            if clusters[i][0] in clusters[j]:
                clusters.pop(i)
                pop = True
                break
        if pop:
            continue

        todo = []
        if flight_data:
          runningRange1 = 1000 # some big number
        else:
          runningRange1 = clusters[i][0]
          

        for j in range(clusters[i][0]):
            if j in dict_edges:
                if clusters[i][0] in dict_edges[j] and j not in clusters[i]:
                    clusters[i].append(j)
                    todo.append(j)

        if clusters[i][0] in dict_edges:
            for key in dict_edges[clusters[i][0]]:
                todo.append(key)
                clusters[i].append(key)

        while len(todo) > 0:
            first = todo.pop()
            if flight_data:
              runningRange2 = 1000 # some big number
            else:
              runningRange2 = first

            for k in range(runningRange2):
                if k in dict_edges:
                    if first in dict_edges[k] and k not in clusters[i]:
                        clusters[i].append(k)
                        todo.append(k)

            if first in dict_edges:
                for key in dict_edges[first]:
                    if key not in clusters[i]:
                        clusters[i].append(key)
                        todo.append(key)
        i += 1

    for i in range(len(clusters)):
        clusters[i] = sorted(clusters[i])

    return clusters


class Plotter:
    def __init__(self, vertex_coordinates, name_dataset, file_loc):
        self.vertex_coordinates = vertex_coordinates
        self.name_dataset = name_dataset
        self.file_loc = file_loc
        self.round = 0
        self.colors = ['blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown',
                       'blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown','blue', 'red', 'green', 'purple', 'yellow', 'darkorange', 'dodgerblue', 'deeppink', 'khaki', 'purple', 'springgreen', 'tomato', 'slategray', 'forestgreen', 'mistyrose', 'mediumorchid','rebeccapurple', 'lavender', 'cornflowerblue', 'lightseagreen', 'brown']
        self.colors2 = ['#663399',
                    '#ff0000',
                    '#b22222',
                    '#4682b4',
                    '#663399',
                    '#d2691e',
                    '#8b008b',
                    '#0000cd',
                    '#00ff7f',
                    '#a0522d',
                    '#a52a2a',
                    '#2e8b57',
                    '#228b22',
                    '#191970',
                    '#006400',
                    '#8b0000',
                    '#808000',
                    '#483d8b',
                    '#b22222',
                    '#5f9ea0',
                    '#778899',
                    '#3cb371',
                    '#bc8f8f',
                    '#663399',
                    '#b8860b',
                    '#bdb76b',
                    '#008b8b',
                    '#cd853f',
                    '#00ff00',
                    '#d2691e',
                    '#9acd32',
                    '#20b2aa',
                    '#cd5c5c',
                    '#00008b',
                    '#4b0082',
                    '#32cd32',
                    '#daa520',
                    '#8fbc8f',
                    '#8b008b',
                    '#b03060',
                    '#66cdaa',
                    '#9932cc',
                    '#ff0000',
                    '#ff4500',
                    '#00ced1',
                    '#ff8c00',
                    '#ffa500',
                    '#ffd700',
                    '#6a5acd',
                    '#ffff00',
                    '#c71585',
                    '#0000cd',
                    '#7cfc00',
                    '#deb887',
                    '#40e0d0',
                    '#00ff00',
                    '#9400d3',
                    '#ba55d3',
                    '#00fa9a',
                    '#8a2be2',
                    '#00ff7f',
                    '#4169e1',
                    '#e9967a',
                    '#dc143c',
                    '#00ffff',
                    '#00bfff',
                    '#f4a460',
                    '#9370db',
                    '#0000ff',
                    '#a020f0',
                    '#f08080',
                    '#adff2f',
                    '#ff6347',
                    '#da70d6',
                    '#d8bfd8',
                    '#b0c4de',
                    '#ff7f50',
                    '#ff00ff',
                    '#1e90ff',
                    '#db7093',
                    '#f0e68c',
                    '#fa8072',
                    '#eee8aa',
                    '#ffff54',
                    '#6495ed',
                    '#dda0dd',
                    '#90ee90',
                    '#add8e6',
                    '#87ceeb',
                    '#ff1493',
                    '#7b68ee',
                    '#ffa07a',
                    '#afeeee',
                    '#ee82ee',
                    '#87cefa',
                    '#7fffd4',
                    '#ffe4b5',
                    '#ffdab9',
                    '#ff69b4',
                    '#ffb6c1'
                    ]
        self.machine_string = "{}_round_{}_machine_".format(self.name_dataset, self.round)

    def update_string(self):
        self.machine_string = "{}_round_{}_machine_".format(self.name_dataset, self.round)

    def set_dataset(self, name_dataset):
        self.name_dataset = name_dataset

    def set_vertex_coordinates(self, vertex_coordinates):
        self.vertex_coordinates = vertex_coordinates

    def set_file_loc(self, file_loc):
        self.file_loc = file_loc

    def reset_round(self):
        self.round = 0

    def next_round(self):
        self.round += 1
        self.update_string()

    def plot_mst_2d(self, mst, intermediate=False, plot_cluster=False, plot_num_machines=0, num_clusters=2, basic_dataset = True):
        x = []
        y = []
        c = []
        area = []

        for i in range(len(self.vertex_coordinates)):
            x.append(float(self.vertex_coordinates[i][0]))
            y.append(float(self.vertex_coordinates[i][1]))
            area.append(0.1)
            c.append('k')

        if intermediate:
            if not basic_dataset:
              map_total = folium.Map(tiles='cartodbpositron', width=1920, height=1080)

            if plot_num_machines > 0:
                cnt = 0
                for m in mst:
                    plt.scatter(x, y, c=c, s=area)

                    for i in range(len(m)):
                        linex = [float(x[int(m[i][0])]), float(x[int(m[i][1])])]
                        liney = [float(y[int(m[i][0])]), float(y[int(m[i][1])])]
                        plt.plot(linex, liney, self.colors[cnt])
                        if not basic_dataset:
                          folium.PolyLine([ ( float(x[int(m[i][0])]), float(y[int(m[i][0])]) ) , ( float(x[int(m[i][1])]), float(y[int(m[i][1])]) )], color=self.colors2[cnt], weight=2, opacity=0.8).add_to(map_total)

                    cnt = (cnt + 1) % len(self.colors)
                    filename = self.file_loc + self.machine_string + '{}'.format(cnt)
                    if not basic_dataset:
                      map_total.save( filename + ".html")
                    plt.savefig(filename, dpi='figure')
                    plt.clf()
                    if cnt >= plot_num_machines:
                        break

            cnt = 0
            for m in mst:
                for i in range(len(m)):
                    linex = [float(x[int(m[i][0])]), float(x[int(m[i][1])])]
                    liney = [float(y[int(m[i][0])]), float(y[int(m[i][1])])]
                    plt.plot(linex, liney, self.colors[cnt])
                    if not basic_dataset:
                      folium.PolyLine([ ( float(x[int(m[i][0])]), float(y[int(m[i][0])]) ) , ( float(x[int(m[i][1])]), float(y[int(m[i][1])]) )], color=self.colors2[cnt], weight=2, opacity=0.8).add_to(map_total)

                cnt = (cnt + 1) % len(self.colors)
            filename = self.file_loc + self.machine_string + 'all'

            if not basic_dataset:
              map_total.save( filename + ".html")
            plt.savefig(filename, dpi='figure')
            plt.clf()
        elif plot_cluster:
            if not basic_dataset:
              map_us = folium.Map(tiles='cartodbpositron', width=1920, height=1080)

            edges = sorted(mst, key=get_key, reverse=True)

            removed_edges = []
            clusters = []
            for i in range(num_clusters - 1):
                edge = edges.pop(0)
                removed_edges.append(edge)
                clusters.append([edge[0]])
                clusters.append([edge[1]])
                linex = [float(x[edge[0]]), float(x[edge[1]])]
                liney = [float(y[edge[0]]), float(y[edge[1]])]
                plt.plot(linex, liney, "k")
                if not basic_dataset:
                  folium.PolyLine([ ( float(x[edge[0]]), float(y[edge[0]]) ) , ( float(x[edge[1]]), float(y[edge[1]]) )], color='black', weight=1, opacity=0.8).add_to(map_us)

            dict_edges = dict()
            for edge in edges:
                if edge[0] in dict_edges:
                    dict_edges[edge[0]].append(edge[1])
                else:
                    dict_edges[edge[0]] = [edge[1]]


            clusters = create_clusters(clusters, dict_edges, flight_data=not basic_dataset)

            x_cluster = []
            y_cluster = []
            c_cluster = []
            area_cluster = []
            
            for i in range(len(clusters)):
                for vertex in clusters[i]:
                    if not basic_dataset: 
                      folium.CircleMarker(location=(float(self.vertex_coordinates[vertex][0]), float(self.vertex_coordinates[vertex][1])), radius=1,color=self.colors2[i], fill_color=self.colors2[i]).add_to(map_us) 
                      folium.CircleMarker(location=(float(self.vertex_coordinates[vertex][0]), float(self.vertex_coordinates[vertex][1])), radius=1,color=self.colors2[i], fill_color=self.colors2[i]).add_to(map_us) 

                    x_cluster.append(float(self.vertex_coordinates[vertex][0]))
                    y_cluster.append(float(self.vertex_coordinates[vertex][1]))

                    area_cluster.append(0.2)
                    c_cluster.append(self.colors[i])

            plt.scatter(x_cluster, y_cluster, c=c_cluster, s=area_cluster)

            for i in range(len(mst)):
                if mst[i] in removed_edges:
                    continue
                    
                linex = [float(x[int(mst[i][0])]), float(x[int(mst[i][1])])]
                liney = [float(y[int(mst[i][0])]), float(y[int(mst[i][1])])]

                for j in range(len(clusters)):
                    if mst[i][0] in clusters[j]:
                        plt.plot(linex, liney, c=self.colors[j])
                        if not basic_dataset:
                          folium.PolyLine([ ( float(x[int(mst[i][0])]), float(y[int(mst[i][0])]) ) , ( float(x[int(mst[i][1])]), float(y[int(mst[i][1])]) )], color=self.colors2[j], weight=2, opacity=0.9).add_to(map_us)

            filename = self.file_loc + self.name_dataset + '_clusters'
            if not basic_dataset:
              map_us.save( filename + ".html")

            plt.savefig(filename, dpi='figure')
            plt.clf()

        else:
          # Plot the MST
          if not basic_dataset:
              map_us = folium.Map(tiles='cartodbpositron', width=1920, height=1080)

          for i in range(len(mst)):
              linex = [float(x[int(mst[i][0])]), float(x[int(mst[i][1])])]
              liney = [float(y[int(mst[i][0])]), float(y[int(mst[i][1])])]
              plt.plot(linex, liney)
              if not basic_dataset:
                folium.PolyLine([ ( float(x[int(mst[i][0])]), float(y[int(mst[i][0])]) ) , ( float(x[int(mst[i][1])]), float(y[int(mst[i][1])]) )], color=self.colors2[i % 100], weight=2, opacity=0.9).add_to(map_us)
                
          filename = self.file_loc + self.name_dataset + '_final'
          if not basic_dataset:
              map_us.save( filename + ".html")
          plt.savefig(filename, dpi='figure')
          plt.clf()


    def plot_cluster(self, yhat, final, vertex_coordinates):
        clusters = set()
        for v in yhat:
            clusters.add(v)
        color_ids = []
        for item in clusters:
            color_ids.append(item)
        x = []
        y = []
        n = len(vertex_coordinates)
        c = ['k'] * n
        area = [0.1] * n
        for x_c, y_c in vertex_coordinates:
            x.append(float(x_c))
            y.append(float(y_c))
        plt.scatter(x, y, c=c, s=area)
        for i in range(n):
            cluster = yhat[i]
            color = self.colors[0]
            for j in range(len(color_ids)):
                if cluster == color_ids[j]:
                    color = self.colors[j % len(self.colors)]
                    break
            linex = [x[i], x[final[i]]]
            liney = [y[i], y[final[i]]]
            plt.plot(linex, liney, color)
        # plt.show()
        filename = self.file_loc + self.name_dataset + str(self.round)
        plt.savefig(filename, dpi='figure')
        plt.clf()

def get_clustering_data():
    """
    Retrieves all toy datasets from sklearn
    :return: circles, moons, blobs datasets.
    """
    n_samples = 1500
    noisy_circles = make_circles(n_samples=n_samples, factor=.5,
                                 noise=0.05)
    noisy_moons = make_moons(n_samples=n_samples, noise=0.05)
    blobs = make_blobs(n_samples=n_samples, random_state=8)
    no_structure = np.random.rand(n_samples, 2), None

    # Anisotropicly distributed data
    random_state = 170
    X, y = make_blobs(n_samples=4000, random_state=random_state)
    transformation = [[0.6, -0.6], [-0.4, 0.8]]
    X_aniso = np.dot(X, transformation)
    aniso = (X_aniso, y)

    # blobs with varied variances
    varied = make_blobs(n_samples=4000,
                        cluster_std=[1.0, 2.5, 0.5],
                        random_state=random_state)

    plt.figure(figsize=(9 * 2 + 3, 13))
    plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.95, wspace=.05,
                        hspace=.01)

    swiss_roll = make_swiss_roll(n_samples, noise=0.05)

    s_shape = make_s_curve(n_samples, noise=0.05)

    temp = np.append(noisy_moons[0], [[2, 1.5]], axis=0)
    noisy_moons = list(noisy_moons)
    noisy_moons[0] = temp

    datasets = [
        # (noisy_circles, {'damping': .77, 'preference': -240,
        #                  'quantile': .2, 'n_clusters': 2,
        #                  'min_samples': 20, 'xi': 0.25}),
        (noisy_moons, {'damping': .75, 'preference': -220, 'n_clusters': 2}),
        # (varied, {'eps': .18, 'n_neighbors': 2,
        #           'min_samples': 5, 'xi': 0.035, 'min_cluster_size': .2}),
        # (aniso, {'eps': .15, 'n_neighbors': 2,
        #          'min_samples': 20, 'xi': 0.1, 'min_cluster_size': .2}),
        # (blobs, {}),
        # (no_structure, {}),
        # (swiss_roll, {}),
        # (s_shape, {})
        ]

    return datasets

def create_distance_matrix(dataset):
    """
    Creates the distance matrix for a dataset with only vertices. Also adds the edges to a dict.
    :param dataset: dataset without edges
    :return: distance matrix, a dict of all edges and the total number of edges
    """
    vertices = []
    size = 0
    three_d = False
    for line in dataset:
        if len(line) == 2:
            vertices.append([line[0], line[1]])
        elif len(line) == 3:
            vertices.append([line[0], line[1], line[2]])
            three_d = True
    if three_d:
        dict = {}
        for i in range(len(dataset)):
            dict2 = {}
            for j in range(i + 1, len(dataset)):
                dict2[j] = np.sqrt(np.sum(np.square(dataset[i] - dataset[j])))
                size += 1
            dict[i] = dict2

    else:
        d_matrix = scipy.spatial.distance_matrix(vertices, vertices, threshold=1000000)
        dict = {}
        # Run with less edges
        for i in range(len(d_matrix)):
            dict2 = {}
            for j in range(i, len(d_matrix)):
                if i != j:
                    size += 1
                    dict2[j] = d_matrix[i][j]
            dict[i] = dict2
    return dict, size, vertices


def partion_vertices(vertices, k):
    """
    Partitioning of the vertices in k smaller subsets (creates a partitioning twice
    :param vertices: all vertices
    :param k: number of subsets that need to be created
    :return: the partitioning in list format
    """
    U = []
    V = []
    random.shuffle(vertices)
    verticesU = vertices.copy()
    random.shuffle(vertices)
    verticesV = vertices.copy()
    for i in range(len(vertices)):
        if i < k:
            U.append({verticesU[i]})
            V.append({verticesV[i]})
        else:
            U[i % k].add(verticesU[i])
            V[i % k].add(verticesV[i])
    return U, V

def get_key(item):
    """
    returns the sorting criteria for the edges. All edges are sorted from small to large values
    :param item: one item
    :return: returns the weight of the edge
    """
    return item[2]

def find_mst(U, V, E):
    """
    finds the mst of graph G = (U union V, E)
    :param U: vertices U
    :param V: vertices V
    :param E: edges of the graph
    :return: the mst and edges not in the mst of the graph
     """
    vertices = set()
    for v in V:
        vertices.add(v)
    for u in U:
        vertices.add(u)
    E = sorted(E, key=get_key)
    connected_component = set()
    mst = []
    remove_edges = set()
    while len(mst) < len(vertices) - 1 and len(connected_component) < len(vertices):
        if len(E) == 0:
            break
        change = False
        i = 0
        while i < len(E):
            if len(connected_component) == 0:
                connected_component.add(E[i][0])
                connected_component.add(E[i][1])
                mst.append(E[i])
                change = True
                E.remove(E[i])
                break
            else:
                if E[i][0] in connected_component:
                    if E[i][1] in connected_component:
                        remove_edges.add(E[i])
                        E.remove(E[i])
                    else:
                        connected_component.add(E[i][1])
                        mst.append(E[i])
                        E.remove(E[i])
                        change = True
                        break
                elif E[i][1] in connected_component:
                    if E[i][0] in connected_component:
                        remove_edges.add(E[i])
                        E.remove(E[i])
                    else:
                        connected_component.add(E[i][0])
                        mst.append(E[i])
                        E.remove(E[i])
                        change = True
                        break
                else:
                    i += 1
        if not change:
            if len(E) != 0:
                connected_component.add(E[0][0])
                connected_component.add(E[0][1])
                mst.append(E[0])
                E.remove(E[0])
    for edge in E:
        remove_edges.add(edge)
    if len(mst) != len(vertices) - 1 or len(connected_component) != len(vertices):
        print('Warning: parition cannot have a full MST! Missing edges to create full MST.')
        # print('Error: MST found cannot be correct \n Length mst: ', len(mst), '\n Total connected vertices: ',
        #       len(connected_component), '\n Number of vertices: ', len(vertices))
    return mst, remove_edges

def get_edges(U, V, E):
    """
    :param U: subset of vertices (u_j)
    :param V: subset of vertices (v_i)
    :param E: all edges of the whole graph
    :return: all edges that are part of the graph u_j U v_j
    """
    edges = set()
    for node1 in U:
        for node2 in V:
            if node1 in E:
                if node2 in E[node1]:
                    edges.add((node1, node2, E[node1][node2]))
                elif node2 in E:
                    if node1 in E[node2]:
                        edges.add((node2, node1, E[node2][node1]))
            elif node2 in E:
                if node1 in E[node2]:
                    edges.add((node2, node1, E[node2][node1]))
    edge_list = []
    for edge in edges:
        edge_list.append(edge)
    return U, V, edge_list


def reduce_edges(vertices, E, c, epsilon):
    """
    Uses PySpark to distribute the computation of the MSTs,
    Randomly partition the vertices twice in k subsets (U = {u_1, u_2, .., u_k}, V = {v_1, v_2, .., v_k})
    For every intersection between U_i and V_j, create the subgraph and find the MST in this graph
    Remove all edges from E that are not part of the MST in the subgraph
    :param vertices: vertices in the graph
    :param E: edges of the graph
    :param c: constant
    :param epsilon:
    :return:The reduced number of edges
    """
    conf = SparkConf().setAppName('MST_Algorithm')
    sc = SparkContext.getOrCreate(conf=conf)

    n = len(vertices)
    k = math.ceil(n ** ((c - epsilon) / 2))
    print("k: ", k)
    U, V = partion_vertices(vertices, k)

    rddUV = sc.parallelize(U).cartesian(sc.parallelize(V)).map(lambda x: get_edges(x[0], x[1], E)).map(
        lambda x: (find_mst(x[0], x[1], x[2])))
    both = rddUV.collect()

    mst = []
    removed_edges = set()
    for i in range(len(both)):
        mst.append(both[i][0])
        for edge in both[i][1]:
            removed_edges.add(edge)

    sc.stop()
    return mst, removed_edges


def remove_edges(E, removed_edges):
    """
    Removes the edges, which are removed when generating msts
    :param E: current edges
    :param removed_edges: edges to be removed
    :return: return the updated edge dict
    """
    for edge in removed_edges:
        if edge[1] in E[edge[0]]:
            del E[edge[0]][edge[1]]
    return E


def create_mst(V, E, epsilon, size, vertex_coordinates, plot_intermediate=False, plotter=None):
    """
    Creates the mst of the graph G = (V, E).
    As long as the number of edges is greater than n ^(1 + epsilon), the number of edges is reduced
    Then the edges that needs to be removed are removed from E and the size is updated.
    :param plotter: class to plot graphs
    :param V: Vertices
    :param E: edges
    :param epsilon:
    :param size: number of edges
    :param plot_intermediate: boolean to indicate if intermediate steps should be plotted
    :param vertex_coordinates: coordinates of vertices
    :return: returns the reduced graph with at most np.power(n, 1 + epsilon) edges
    """
    n = len(V)
    c = math.log(size / n, n)
    print("C", c)
    total_runs = 0
    while size > np.power(n, 1 + epsilon):
        total_runs += 1
        if plotter is not None:
            plotter.next_round()
        mst, removed_edges = reduce_edges(V, E, c, epsilon)
        if plot_intermediate and plotter is not None:
            if len(vertex_coordinates[0]) > 2:
                plotter.plot_mst_3d(mst, intermediate=True, plot_cluster=False, plot_num_machines=1)
            else:
                plotter.plot_mst_2d(mst, intermediate=True, plot_cluster=False, plot_num_machines=1)
        E = remove_edges(E, removed_edges)
        print('Total edges removed in this iteration', len(removed_edges))
        size = size - len(removed_edges)
        print('New total of edges: ', size)
        c = (c - epsilon) / 2

    # Now the number of edges is reduced and can be moved to a single machine
    V = set(range(n))
    items = E.items()  # returns [(x, {y : 1})]
    edges = []
    for item in items:
        items2 = item[1].items()
        for item2 in items2:
            edges.append((item[0], item2[0], item2[1]))
    mst, removed_edges = find_mst(V, V, edges)
    print("#####\n\nTotal runs: ", total_runs, "\n\n#####")
    return mst

In [55]:
def main():
    """
    For every dataset, it creates the mst and plots the clustering
    """
    start_time = datetime.now()
    print('Starting time:', start_time)

    datasets = get_clustering_data()
    names_datasets = ['circles','moons']

    num_clusters = [2,2,3, 3, 2, 2, 2]
    cnt = 0
    time = []
    file_location = 'Results/Test/'
    plotter = Plotter(None, None, file_location)
    data_reader = DataReader()
    for dataset in datasets:
        if cnt < 0:
            cnt += 1
            continue
        timestamp = datetime.now()
        print('Start creating Distance Matrix...')
        E, size, vertex_coordinates = create_distance_matrix(dataset[0][0])
        plotter.set_vertex_coordinates(vertex_coordinates)
        plotter.set_dataset(names_datasets[cnt])
        plotter.update_string()
        plotter.reset_round()
        V = list(range(len(vertex_coordinates)))
        print('Size dataset: ', len(vertex_coordinates))
        print('Created distance matrix in: ', datetime.now() - timestamp)
        print('Start creating MST...')
        timestamp = datetime.now()
        mst = create_mst(V, E, epsilon=1/8, size=size, vertex_coordinates=vertex_coordinates,
                         plot_intermediate=True, plotter=plotter)
        print('Found MST in: ', datetime.now() - timestamp)
        time.append(datetime.now() - timestamp)
        print('Start creating plot of MST...')
        timestamp = datetime.now()
        plotter.plot_mst_2d(mst, intermediate=False, plot_cluster=True, num_clusters=2)
        plotter.plot_mst_2d(mst, intermediate=False, plot_cluster=False, num_clusters=2)
        print('Created plot of MST in: ', datetime.now() - timestamp)
        cnt += 1
        
main()

Starting time: 2022-12-12 22:02:54.793887
Start creating Distance Matrix...
Size dataset:  1501
Created distance matrix in:  0:00:00.622319
Start creating MST...
C 0.905137495141123
k:  18
Total edges removed in this iteration 1109255
New total of edges:  16495
k:  3
Total edges removed in this iteration 13855
New total of edges:  2640
#####

Total runs:  2 

#####
Found MST in:  0:01:36.731487
Start creating plot of MST...
Created plot of MST in:  0:00:04.085705


<Figure size 1512x936 with 0 Axes>

In [56]:
# Read the .csv file
def read_data_set_from_csvfile(file_location):
    edges = {}
    vertices = []
    size = 0
    with open(file_location) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
          if line_count == 0:
            line_count += 1
            continue
          
          line_count += 1

          v1_id = int(row[1])
          v2_id = int(row[4])

          if not [float(row[2]), float(row[3])] in vertices:
            vertices.append([float(row[2]), float(row[3])])

          if not [float(row[5]), float(row[6])] in vertices:
            vertices.append([float(row[5]), float(row[6])])

          if v1_id in edges:
              edges[v1_id][v2_id] = float(row[7])
              size += 1
          else:
              edges[v1_id] = {v2_id: float(row[7])}
              size += 1

    return vertices, size, edges

vertices, size, edges = read_data_set_from_csvfile('Results/flight_data.csv')

file_location = 'Results/Flights/'
plotter = Plotter(None, None, file_location)
timestamp = datetime.now()
E, size, vertex_coordinates = edges, size, vertices
plotter.set_vertex_coordinates(vertex_coordinates)
plotter.set_dataset('flight_data_2')
plotter.update_string()
plotter.reset_round()
V = list(range(len(vertex_coordinates)))
print('Size dataset: ', len(vertex_coordinates))
print('Created distance matrix in: ', datetime.now() - timestamp)
print('Start creating MST...')
timestamp = datetime.now()
mst = create_mst(V, E, epsilon=1/8, size=size, vertex_coordinates=vertex_coordinates, plot_intermediate=True, plotter=plotter)
print('Start creating plot of MST...')

plotter.plot_mst_2d(mst, intermediate=False, plot_cluster=True, num_clusters=7, basic_dataset=False)
print('Did everything in', datetime.now() - timestamp)

Size dataset:  358
Created distance matrix in:  0:00:00.003825
Start creating MST...
C 0.4772935517712418
k:  3
Total edges removed in this iteration 4574
New total of edges:  1353
k:  2
Total edges removed in this iteration 659
New total of edges:  694
#####

Total runs:  2 

#####
Start creating plot of MST...
Did everything in 0:00:08.423893


<Figure size 432x288 with 0 Axes>